In [1]:
%pip list

Package                Version
---------------------- -------
absl-py                0.7.1  
asn1crypto             0.24.0 
astor                  0.8.0  
attrs                  19.1.0 
backcall               0.1.0  
bleach                 3.1.0  
cloudpickle            1.1.1  
cryptography           2.1.4  
cycler                 0.10.0 
decorator              4.4.0  
defusedxml             0.6.0  
entrypoints            0.3    
enum34                 1.1.6  
gast                   0.2.2  
google-pasta           0.1.7  
grpcio                 1.21.1 
h5py                   2.9.0  
idna                   2.6    
ipykernel              5.1.1  
ipython                7.5.0  
ipython-genutils       0.2.0  
ipywidgets             7.4.2  
jedi                   0.13.3 
Jinja2                 2.10.1 
jsonschema             3.0.1  
jupyter                1.0.0  
jupyter-client         5.2.4  
jupyter-console        6.0.0  
jupyter-core           4.5.0  
jupyter-http-over-ws   0.0.6  
Keras-Ap

In [ ]:
import numpy as np
import os
import random
import tensorflow as tf
import imageio

In [ ]:
def get_images(paths, labels, nb_samples=None, shuffle=True):
    """
    Takes a set of character folders and labels and returns paths to image files
    paired with labels.
    Args:
        paths: A list of character folders
        labels: List or numpy array of same length as paths
        nb_samples: Number of images to retrieve per character
    Returns:
        List of (label, image_path) tuples
    """
    if nb_samples is not None:
        sampler = lambda x: random.sample(x, nb_samples)
    else:
        sampler = lambda x: x
    images_labels = [(i, os.path.join(path, image))
                     for i, path in zip(labels, paths)
                     for image in sampler(os.listdir(path))]
    if shuffle:
        random.shuffle(images_labels)
    return images_labels

In [ ]:
def image_file_to_array(filename, dim_input):
    """
    Takes an image path and returns numpy array
    Args:
        filename: Image filename
        dim_input: Flattened shape of image
    Returns:
        1 channel image
    """
    image = imageio.imread(filename)
    image = image.reshape([dim_input])
    image = image.astype(np.float32) / 255.0
    image = 1.0 - image
    return image

In [ ]:
class DataGenerator(object):
    """
    Data Generator capable of generating batches of Omniglot data.
    A "class" is considered a class of omniglot digits.
    """

    def __init__(self, num_classes, num_samples_per_class, config={}):
        """
        Args:
            num_classes: Number of classes for classification (K-way)
            num_samples_per_class: num samples to generate per class in one batch
            batch_size: size of meta batch size (e.g. number of functions)
        """
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes

        data_folder = config.get('data_folder', './omniglot_resized')
        self.img_size = config.get('img_size', (28, 28))

        self.dim_input = np.prod(self.img_size)
        self.dim_output = self.num_classes

        character_folders = [os.path.join(data_folder, family, character)
                             for family in os.listdir(data_folder)
                             if os.path.isdir(os.path.join(data_folder, family))
                             for character in os.listdir(os.path.join(data_folder, family))
                             if os.path.isdir(os.path.join(data_folder, family, character))]

        random.seed(1)
        random.shuffle(character_folders)
        num_val = 100
        num_train = 1100
        self.metatrain_character_folders = character_folders[: num_train]
        self.metaval_character_folders = character_folders[
            num_train:num_train + num_val]
        self.metatest_character_folders = character_folders[
            num_train + num_val:]

    def sample_batch(self, batch_type, batch_size):
        """
        Samples a batch for training, validation, or testing
        Args:
            batch_type: train/val/test
        Returns:
            A a tuple of (1) Image batch and (2) Label batch where
            image batch has shape [B, K, N, 784] and label batch has shape [B, K, N, N]
            where B is batch size, K is number of samples per class, N is number of classes
        """
        if batch_type == "train":
            folders = self.metatrain_character_folders
        elif batch_type == "val":
            folders = self.metaval_character_folders
        else:
            folders = self.metatest_character_folders

        #############################
        #### YOUR CODE GOES HERE ####
        labels_images=[]

        for i in range(batch_size):
            paths=random.sample(folders, self.num_classes)
            labels=np.eye(self.num_classes).tolist()
            for j in range(self.num_samples_per_class):
                labels_images.extend(get_images(paths, labels, 1, shuffle=True))

        all_image_batches=[image_file_to_array(label_image[1],784) for label_image in labels_images] 
        all_label_batches=[label_image[0] for label_image in labels_images]
        all_image_batches=np.reshape(all_image_batches,(batch_size, self.num_samples_per_class, self.num_classes, 784))
        all_label_batches=np.reshape(all_label_batches, (batch_size, self.num_samples_per_class, self.num_classes, self.num_classes))
        #############################

        return all_image_batches, all_label_batches

In [ ]:
def test_data_generator(batch_size, num_classes, num_samples_per_class):
    """
        test the DataGenerator class's sample_batch function
    Args:
        batch size, number of classes, number of samples per class
    Returns:
        nothing, prints images
    """
    data_generator = DataGenerator(num_classes, num_samples_per_class + 1)
    print('Testing data generator for ', num_samples_per_class, 'shot ', num_classes, \
          'way meta-learner with a batch size', batch_size)
    train_image_batch, train_label_batch = data_generator.sample_batch('train', batch_size)
    print('train image batch shape is ', train_image_batch.shape)
    print('train label batch shape is ', train_label_batch.shape)
    val_image_batch, val_label_batch = data_generator.sample_batch('val', batch_size)
    print('val image batch shape is ', val_image_batch.shape)
    print('val label batch shape is ', val_label_batch.shape)
    test_image_batch, test_label_batch = data_generator.sample_batch('test', batch_size)
    print('test image batch shape is ', test_image_batch.shape)
    print('test label batch shape is ', test_label_batch.shape)
    return

test_data_generator(16,5,1)


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def view_omniglot_imgfiles(n_characters, n_images_pchar, data_folder = './omniglot_resized'):
    """
        view random multiple omniglot image files, for demo purposes
    Args:
        number of characters, number of images per character, data folder [ex. ./omniglot_resized]
    Returns:
        nothing, prints images
    """
    character_folders = [os.path.join(data_folder, family, character)
                             for family in os.listdir(data_folder)
                             if os.path.isdir(os.path.join(data_folder, family))
                             for character in os.listdir(os.path.join(data_folder, family))
                             if os.path.isdir(os.path.join(data_folder, family, character))]
    
    paths=random.sample(character_folders, n_characters)
    labels=np.eye(n_characters).tolist()
    labels_images=get_images(paths, labels, n_images_pchar, shuffle=False)    
    image_files =[label_image[1] for label_image in labels_images]

    for image_file in image_files:
        img=Image.open(image_file)
        plt.figure()
        plt.imshow(img)
    return

view_omniglot_imgfiles(10, 2)


In [ ]:
import numpy as np
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from load_data import DataGenerator
#from tensorflow.compat.v1 import flags
flags = tf.app.flags 
FLAGS = tf.app.flags.FLAGS
from tensorflow.keras import layers

#FLAGS = flags.FLAGS

flags.DEFINE_integer(
    'num_classes', 5, 'number of classes used in classification (e.g. 5-way classification).')

flags.DEFINE_integer('num_samples', 1,
                     'number of examples used for inner gradient update (K for K-shot learning).')

flags.DEFINE_integer('meta_batch_size', 16,
                     'Number of N-way classification tasks per batch')



In [ ]:
def loss_function(preds, labels):
    """
    Computes MANN loss
    Args:
        preds: [B, K+1, N, N] network output
        labels: [B, K+1, N, N] labels
    Returns:
        scalar loss
    """
    #############################
    #### YOUR CODE GOES HERE ####
    pass
    #############################

class MANN(tf.keras.Model):

    def __init__(self, num_classes, samples_per_class):
        super(MANN, self).__init__()
        self.num_classes = num_classes
        self.samples_per_class = samples_per_class
        self.layer1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.layer2 = tf.keras.layers.LSTM(num_classes, return_sequences=True)

    def call(self, input_images, input_labels):
        """
        MANN
        Args:
            input_images: [B, K+1, N, 784] flattened images
            labels: [B, K+1, N, N] ground truth labels
        Returns:
            [B, K+1, N, N] predictions
        """
        #############################
        #### YOUR CODE GOES HERE ####
        pass
        #############################
        return out
    


In [ ]:
ims = tf.placeholder(tf.float32, \
                     shape=(None, FLAGS.num_samples + 1, FLAGS.num_classes, 784))
labels = tf.placeholder(tf.float32, \
                        shape=(None, FLAGS.num_samples + 1, FLAGS.num_classes, FLAGS.num_classes))

data_generator = DataGenerator(
    FLAGS.num_classes, FLAGS.num_samples + 1)

o = MANN(FLAGS.num_classes, FLAGS.num_samples + 1)
out = o(ims, labels)

loss = loss_function(out, labels)
optim = tf.train.AdamOptimizer(0.001)
optimizer_step = optim.minimize(loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())

    for step in range(50000):
        i, l = data_generator.sample_batch('train', FLAGS.meta_batch_size)
        feed = {ims: i.astype(np.float32), labels: l.astype(np.float32)}
        _, ls = sess.run([optimizer_step, loss], feed)

        if step % 100 == 0:
            print("*" * 5 + "Iter " + str(step) + "*" * 5)
            i, l = data_generator.sample_batch('test', 100)
            feed = {ims: i.astype(np.float32),
                    labels: l.astype(np.float32)}
            pred, tls = sess.run([out, loss], feed)
            print("Train Loss:", ls, "Test Loss:", tls)
            pred = pred.reshape(
                -1, FLAGS.num_samples + 1,
                FLAGS.num_classes, FLAGS.num_classes)
            pred = pred[:, -1, :, :].argmax(2)
            l = l[:, -1, :, :].argmax(2)
            print("Test Accuracy", (1.0 * (pred == l)).mean())
